In [1]:
!pip install kaggle

In [2]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
kaggle_dictionary = json.load(open('kaggle.json'))
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [9]:

!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 23% 6.00M/25.7M [00:00<00:00, 59.0MB/s]
100% 25.7M/25.7M [00:00<00:00, 140MB/s] 


In [10]:

!ls


imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [11]:
# unzip the dataset file
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
   zip_ref.extractall()

In [12]:

data  = pd.read_csv('/content/IMDB Dataset.csv')

In [13]:

data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [14]:

data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [15]:

data.replace({'sentiment' : {'positive':1, 'negative':0}}, inplace=True)

<ipython-input-15-01bf67c43f00>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'sentiment' : {'positive':1, 'negative':0}}, inplace=True)


In [16]:
data.shape

(50000, 2)

In [17]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [18]:
train_data.shape

(40000, 2)

In [19]:
test_data.shape

(10000, 2)

In [20]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [21]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [22]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [24]:

# compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)


Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 176s 347ms/step - accuracy: 0.7206 - loss: 0.5373 - val_accuracy: 0.8366 - val_loss: 0.3889
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 345ms/step - accuracy: 0.8377 - loss: 0.3798 - val_accuracy: 0.8530 - val_loss: 0.3482
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 210s 361ms/step - accuracy: 0.8691 - loss: 0.3209 - val_accuracy: 0.8689 - val_loss: 0.3277
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 199s 355ms/step - accuracy: 0.8919 - loss: 0.2765 - val_accuracy: 0.8669 - val_loss: 0.3335
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 357ms/step - accuracy: 0.8979 - loss: 0.2568 - val_accuracy: 0.8554 - val_loss: 0.3549


In [26]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 16s 50ms/step - accuracy: 0.8537 - loss: 0.3535
Test Loss: 0.3538528382778168
Test Accuracy: 0.8543999791145325


In [29]:
y_prediction=model.predict(X_test)
y_prediction

313/313 ━━━━━━━━━━━━━━━━━━━━ 18s 55ms/step


array([[0.21663873],
       [0.8762632 ],
       [0.13440089],
       ...,
       [0.9800461 ],
       [0.0188662 ],
       [0.974568  ]], dtype=float32)

In [27]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [46]:
# example usage
new_review = "The Role of Iron Man can be played better by Captain America"
sentiment = predict_sentiment(new_review)
print(f"Movie Review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Movie Review is: negative
